In [41]:
import tensorflow as tf
import numpy as np
from sklearn.cross_validation import train_test_split
#import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [42]:
credit2 = pd.read_csv("credit4.csv")
credit2.head()

,Unnamed: 0,PAY_0,maximum.delay,AMT1_PBILLIM,BILL_AMT1_cent,AMT3_PBILLIM,AMT1_PPAYBIL,PAY_2,AMT6_PBILLIM,AMT4_PBILLIM,BILL_AMT2_cent,AMT5_PBILLIM,default.payment.next.month
0,1,2,2,0.195650,-0.642490,0.034450,0.000000,2,0.000000,0.000000,-0.647388,0.000000,1
1,2,-1,2,0.022350,-0.659208,0.022350,0.000000,2,0.027175,0.027267,-0.666735,0.028792,1
2,3,0,0,0.324878,-0.298555,0.150656,0.051917,0,0.172767,0.159233,-0.493891,0.166089,0
3,4,0,0,0.939800,-0.057490,0.985820,0.042562,0,0.590940,0.566280,-0.013292,0.579180,0
4,5,-1,0,0.172340,-0.578608,0.716700,0.232099,0,0.382620,0.418800,-0.611308,0.382920,0


In [43]:
X_train, X_test,Y_train,Y_test = train_test_split(credit2.iloc[:,1:12],credit2[['default.payment.next.month']], train_size=0.75)
Xf=np.asarray(X_train)
Yf=np.asarray(Y_train)
Xftes = np.asarray(X_test)
Yftes = np.asarray(Y_test)
X_train.head()

,PAY_0,maximum.delay,AMT1_PBILLIM,BILL_AMT1_cent,AMT3_PBILLIM,AMT1_PPAYBIL,PAY_2,AMT6_PBILLIM,AMT4_PBILLIM,BILL_AMT2_cent,AMT5_PBILLIM
14240,-1,-1,0.020294,-0.557831,0.023454,0.728787,-1,0.026034,0.021700,-0.587591,0.018122
13162,-1,2,0.039000,-0.685038,0.019500,0.499999,2,0.086750,0.039000,-0.680013,0.019500
22057,0,0,0.113900,-0.185186,0.119106,0.053210,0,0.118115,0.121973,-0.140446,0.126312
17950,2,3,0.472645,0.588106,0.473860,0.000000,3,0.518820,0.461840,0.603409,0.509185
13271,-1,0,0.023262,-0.594538,0.109091,0.776867,-1,0.018200,0.050978,-0.610113,0.028997


In [44]:
#declaracion de palabras
input_size=11
output_layer_size=1
#Los placeholders son para los datos de entrada
x = tf.placeholder(tf.float32, [None, input_size])
y_ = tf.placeholder(tf.float32, [None, output_layer_size]) #la y real

#La variables para lo que se va a ir calculando y modificando en el camino
#W_layer1=tf.Variable(tf.random_uniform([input_size,output_layer_size], -1, 1), name="W_layer1")
#b_layer1 = tf.Variable(tf.zeros([output_layer_size]), name="b_layer1")

W_layer1 = tf.Variable(tf.random_uniform([11, 8], -1, 1), 
                       name = "W_layer1")
b_layer1 = tf.Variable(tf.zeros([8]), 
                       name = "b_layer1")

W_layer2 = tf.Variable(tf.random_uniform([8, 6], -1, 1), 
                       name = "W_layer2")
b_layer2 = tf.Variable(tf.zeros([6]), 
                       name = "b_layer2")

W_layer3 = tf.Variable(tf.random_uniform([6, 1], -1, 1), 
                       name = "W_layer3")
b_layer3 = tf.Variable(tf.zeros([1]), 
                       name = "b_layer3")
#operaciones de grafo
hd1 = tf.nn.sigmoid( tf.matmul(x, W_layer1) + b_layer1)
hd2 = tf.nn.sigmoid( tf.matmul(hd1, W_layer2) + b_layer2)
y = tf.nn.sigmoid( tf.matmul(hd2, W_layer3) + b_layer3) 
#y = tf.nn.sigmoid(tf.matmul(x,W_layer1)+b_layer1) #la de nuestra
#lossfn = tf.reduce_mean(tf.reduce_sum((y_-y)**2)) #cuadratico
lossfn= tf.reduce_mean(( (y_ * tf.log(y)) +  
  ((1 - y_) * tf.log(1.0 - y)) ) * -1) # cross entropy
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(lossfn) #ojo si disminuye baja el accuracy 

init = tf.initialize_all_variables()
sess = tf.Session() # tipo de sesion, puede ser interactiva

#correr programa
sess.run(init)
for i in range(30000):
  sess.run(train_step, feed_dict={x: Xf, y_: Yf})

#imprimir alguna variable
print(sess.run(y, feed_dict={x: Xf, y_: Yf}))
print(sess.run(W_layer1, feed_dict={x: Xf, y_: Yf}))

#cuantificar error
correct_prediction = tf.equal(tf.round(y),y_) # aca estams definiendo otras operacioes
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: Xf, y_: Yf})) # las corremos

[[ 0.11818796]
 [ 0.2472993 ]
 [ 0.10926388]
 ..., 
 [ 0.16035308]
 [ 0.49433303]
 [ 0.12947971]]
[[-0.01384391 -0.92913538 -0.03166237  1.20555329  0.76654166 -0.311667
   0.34800565 -0.47539449]
 [ 0.26334962 -0.25767052  1.21993291 -0.81675142  0.90451419 -1.04184282
  -0.54270637 -0.59875476]
 [-0.60025066 -0.16543487  0.22325321  0.10015918 -0.78798956 -0.2396999
   0.37465781  0.77359045]
 [-0.50487483  0.39616662  0.01299628  0.66721809 -0.34401691 -0.74187917
   0.95657933  0.65562999]
 [ 0.054293   -0.00323632  0.93647373 -0.18102285 -0.47776648 -1.0737083
  -0.70369017 -0.01858673]
 [ 0.97894925 -0.05805622 -0.70263225 -0.83793205 -0.0822467  -0.4813745
  -0.67889273  0.59396213]
 [-0.9573698  -1.37963855 -0.21870436 -0.63995069  0.00621243 -0.11647183
   1.03769171 -0.13569385]
 [ 0.03231564  0.37932226 -0.11838102  0.89374131  0.56250846 -0.0622941
  -0.91803926 -0.05485981]
 [ 0.41855291  0.26904932 -0.40413561  0.52423036 -0.07065957 -0.69977999
   0.04140383 -1.04382133]